In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [7]:
def cnn_model_fn(features, labels, mode):
    # Input layer
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])

    # Convolutional layer
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=32,
                             kernel_size=[5, 5], padding='same',
                             activation=tf.nn.relu)
    print(conv1.shape)

    # pooling layer
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                    strides=2)

    # Convolutional and pooling layer
    conv2 = tf.layers.conv2d(inputs=pool1, filters=64,
                             kernel_size=[5, 5], padding='same',
                             activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                    strides=2)
    print(pool2.shape)

    # Dense layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4,
                                training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate loss(for both train and eval modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the training op(for train mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())

        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics(for EVAL mode)
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
def main(unused_argv):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset('mnist')
    train_data = mnist.train.images
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

    # create the estimator(一个用于执行高级模型训练，评估，推理的tensorflow累哦)
    mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                              model_dir='mnist_model')

    # 它将cnn中的softmax层的概率值记录下来
    tensor_to_log = {'probabilities': 'softmax_tensor'}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensor_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x': train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True
    )
    mnist_classifier.train(input_fn=train_input_fn,
                           steps=20000, hooks=[logging_hook])

    # 训练完成后，评估模型
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x': eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False
    )
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [10]:
if __name__ == '__main__':
    tf.app.run()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.


Extracting MNIST-data\train-images-idx3-ubyte.gz


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.


Extracting MNIST-data\train-labels-idx1-ubyte.gz


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data\t10k-images-idx3-ubyte.gz


...

INFO:tensorflow:probabilities = [[0.06004284 0.07568169 0.0952082  0.08950377 0.20698029 0.06767879
  0.07271057 0.07661484 0.08855566 0.1670234 ]
 [0.22785716 0.03545799 0.12965553 0.09928342 0.09073947 0.06817362
  0.06136727 0.06544961 0.05859605 0.1634199 ]
 [0.08261725 0.08740976 0.08863971 0.12097915 0.09813587 0.08305106
  0.14697362 0.0754026  0.11770617 0.09908477]
 [0.03775598 0.18614392 0.09263533 0.1396027  0.06538822 0.08211604
  0.09414554 0.11302615 0.09042355 0.09876247]
 [0.04114872 0.15286848 0.07028625 0.10869269 0.08183466 0.05937139
  0.07257558 0.2011229  0.09619351 0.11590584]
 [0.23084217 0.02868082 0.06977583 0.09296486 0.11444955 0.06154707
  0.11876474 0.11950196 0.08359257 0.07988037]
 [0.31852996 0.02106136 0.09946293 0.12235161 0.03836015 0.08382488
  0.08660471 0.05325685 0.1210067  0.0555408 ]
 [0.06456894 0.15823385 0.09820572 0.12408654 0.08538865 0.08346483
  0.11759979 0.06434099 0.131708   0.0724027 ]
 [0.11427406 0.07790142 0.11939078 0.1144369  0.

INFO:tensorflow:global_step/sec: 6.00132


INFO:tensorflow:probabilities = [[0.05528302 0.06676276 0.05565176 0.16189164 0.10123826 0.10388406
  0.08049945 0.11478131 0.1173289  0.14267881]
 [0.05017559 0.22207898 0.09286582 0.08849926 0.10719279 0.06489986
  0.08060021 0.07357869 0.12468106 0.09542772]
 [0.13523364 0.05142722 0.11423447 0.22425106 0.04113471 0.12386023
  0.06859589 0.06790873 0.10505631 0.06829783]
 [0.09657194 0.08470123 0.10282645 0.1021102  0.1145466  0.07806871
  0.10708769 0.08135185 0.11403319 0.11870208]
 [0.08862424 0.06434754 0.10503849 0.10988323 0.16908884 0.07173084
  0.07021053 0.09735518 0.06857605 0.15514502]
 [0.08377595 0.04440211 0.07386959 0.10795402 0.16739729 0.05566552
  0.07848259 0.15274732 0.07118325 0.16452233]
 [0.065907   0.04198392 0.06564715 0.08728603 0.22596629 0.0875418
  0.10081267 0.07924082 0.07252901 0.17308539]
 [0.04582865 0.07238202 0.07902037 0.26142457 0.04987783 0.06813768
  0.04865216 0.13571583 0.1710912  0.06786972]
 [0.10694333 0.04366295 0.06898176 0.07258625 0.1

INFO:tensorflow:loss = 1.7770765, step = 1101 (16.662 sec)


INFO:tensorflow:probabilities = [[0.04569846 0.11209353 0.06079007 0.10846452 0.07277924 0.12171002
  0.05277605 0.16535598 0.11657824 0.14375399]
 [0.06487404 0.10794359 0.08341712 0.11655419 0.09108455 0.07277618
  0.08840717 0.10086784 0.15078133 0.123294  ]
 [0.0487872  0.03520314 0.05529641 0.14627667 0.11208116 0.10198864
  0.05745726 0.23503946 0.10022634 0.10764372]
 [0.06961892 0.07661057 0.12411173 0.15073773 0.05659758 0.10388619
  0.07416521 0.12791376 0.13210641 0.08425192]
 [0.04796104 0.32033813 0.06635077 0.08351405 0.07405107 0.06063759
  0.05047968 0.08664788 0.11482376 0.09519606]
 [0.05148074 0.07500151 0.09582011 0.12062058 0.11482649 0.09397694
  0.0344726  0.17606395 0.10013955 0.13759744]
 [0.16209278 0.02905121 0.11611984 0.057713   0.10366095 0.07302084
  0.23339325 0.04454768 0.09040551 0.08999493]
 [0.03296508 0.400212   0.10934917 0.05317047 0.07000938 0.03998436
  0.03419093 0.06492461 0.12644753 0.06874643]
 [0.07348564 0.04441205 0.10706591 0.13327588 0.